In [1]:
from pixtrack.utils.pytorch3d_render_utils import (
    render_image, 
    create_colmap_camera,
    create_colmap_image_from_pytorch3d_RT,
    create_look_at_camera_poses
)
from pathlib import Path
import math
from pytorch3d.io import load_objs_as_meshes
import torch
from tqdm import tqdm
import numpy as np
import cv2

In [2]:
def render_dataset_and_get_colmap_images_and_cameras(
    mesh_path, dataset_path, fx, fy, cx, cy, W, H, k1, device, subdivisions=2
):
    Rs, Ts, mesh = create_look_at_poses_for_mesh_2(
        fx, fy, W, H, mesh_path, subdivisions=subdivisions, device=device
    )
    dataset_path.mkdir(parents=True, exist_ok=True)
    mapping_path = dataset_path / "mapping"
    mapping_path.mkdir(parents=True, exist_ok=True)
    colmap_images = {}
    colmap_camera_id = 1
    colmap_camera = create_colmap_camera(W, H, fx, cx, cy, k1)
    colmap_cameras = {colmap_camera_id: colmap_camera}
    for i in tqdm(range(Rs.shape[0])):
        R = Rs[i]
        T = Ts[i]
        image = render_image(mesh, fx, fy, cx, cy, W, H, R.unsqueeze(0), T.unsqueeze(0))
        image = (image[..., :3] * 255.0).astype(np.uint8)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        img_name = ("%d.png" % (i + 1)).zfill(8)
        cv2.imwrite(str(mapping_path / img_name), image)

        colmap_image_name = "mapping/" + img_name
        colmap_image_id = i + 1
        colmap_image = create_colmap_image_from_pytorch3d_RT(
            R, T, colmap_image_name, colmap_image_id, colmap_camera_id
        )
        colmap_images[colmap_image_id] = colmap_image
    return colmap_images, colmap_cameras

In [3]:
def create_look_at_poses_for_mesh_2(
    fx, fy, W, H, mesh_path, subdivisions=2, device=torch.device("cuda:0")
):
    mesh = load_objs_as_meshes([mesh_path], device=device)
    torch.min(mesh.verts_list()[0], dim=0).values
    mesh_min = torch.min(mesh.verts_list()[0], dim=0).values
    mesh_max = torch.max(mesh.verts_list()[0], dim=0).values
    max_dist = torch.sqrt(torch.sum((mesh_max - mesh_min) ** 2))
    radius = float(max_dist) / 2.
    dist = get_camera_distance(radius, fx, fy, W, H)
    Rs, Ts = create_look_at_camera_poses(dist, subdivisions)
    return Rs, Ts, mesh

In [4]:
def get_camera_distance(radius, fx, fy, W, H):
    angle_x = math.atan(W / (fx * 2))
    angle_y = math.atan(H / (fy * 2))
    #fovx = angle_x * 180 / math.pi
    #fovy = angle_y * 180 / math.pi
    d = max(radius / math.sin(angle_x), radius / math.sin(angle_y))
    return d
    

In [5]:
fx = 1.066778e+03
fy = 1.066778e+03
cx = 3.129869e+02
cy = 2.413109e+02
W = 640
H = 480
k1 = 0.0
mesh_path = Path('/data/ycb/models/003_cracker_box/textured.obj')
dataset_path = Path('/data/test/')
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

In [6]:
colmap_images, colmap_cameras = render_dataset_and_get_colmap_images_and_cameras(
        mesh_path, dataset_path, fx, fy, cx, cy, W, H, k1, device
    )

100%|██████████████████████████████████████████████████████████| 162/162 [00:16<00:00,  9.74it/s]


In [ ]:
image = render_image(mesh, fx, fy, cx, cy, W, H, R.unsqueeze(0), T.unsqueeze(0))